In [ ]:
import json
from collections import defaultdict

# 输入文件路径
dataset_file = 'Benchmark/fine_grained.json'
llm2_file = 'Benchmark/license_data/license_llm/evaluation/result/qianwen-5.json'


with open(dataset_file, 'r') as f:
    dataset_data = json.load(f)

with open(llm2_file, 'r') as f:
    llm2_data = json.load(f)

llm2_dict = {
    (entry['license_name'], term_info['term']): term_info
    for entry in llm2_data for term_info in entry['terms']
}

# 存储最终结果
llm2_all = defaultdict(int)
llm2_none = defaultdict(int)
llm2_hall = defaultdict(int)
llm2_diff = defaultdict(int)

# 处理条款
for license_info in dataset_data:
    license_name = license_info['license_name']
    terms = license_info['terms']

    for term_info in terms:
        term = term_info['term']
        marker = term_info['marker']
        llm2_all[term] += 1
        
        llm2_key = (license_name, term)
        llm2_result = llm2_dict[llm2_key]
        response = llm2_result.get('response')
        
        if llm2_result is None:
            print(license_name)
            continue  # 如果没有对应项，则跳过
        
        if isinstance(response, dict):
            response_marker = response.get('marker')
            
            if response_marker is None:
                llm2_hall[term] += 1

            elif response_marker != marker:
                llm2_diff[term] += 1
                
        elif llm2_result == 'None':
            llm2_none[term] += 1
        else:
            llm2_hall[term] += 1
            

order = [
    "Distribute", "Modify", "Commercial Use", "Relicense", "Hold Liable",
    "Use Patent Claims", "Sublicense", "Use Trademark", "Include Copyright",
    "Include License", "Disclose Source", "Give Credit", "Rename", 
    "Contact Author"
]

print('ALL:')
for key in order:
    print(f"{llm2_all.get(key, 0)}")


print('NONE:')
for key in order:
    print(f"{llm2_none.get(key, 0)}")

print("HALL:")
for key in order:
    print(f"{llm2_hall.get(key, 0)}")

print("DIFF:")
for key in order:
    print(f"{llm2_diff.get(key, 0)}")



ALL:
579
571
177
43
429
104
193
210
361
196
275
160
43
55
NONE:
0
0
0
0
0
0
0
0
0
0
0
0
0
0
HALL:
0
0
0
0
0
1
0
1
0
0
1
0
0
0
DIFF:
196
198
102
33
113
75
124
110
78
98
156
61
24
43
